In [5]:
import pandas as pd
import numpy as np
import os

# Define data directory
path = "/Users/rholt/Desktop/personal/VU Spring2025/ResponsibleAI/assignment3"
data_dir = os.path.join(path, "NewB")

# Load datasets
train_file = os.path.join(data_dir, "train_orig.txt")
test_file = os.path.join(data_dir, "test.txt")

# Ensure files exist before loading
if not os.path.exists(train_file) or not os.path.exists(test_file):
    raise FileNotFoundError(f"One or more dataset files are missing in {data_dir}")

# Load train dataset
train_data = pd.read_csv(train_file, sep='\t', header=None, names=['label', 'sentence'])

# Secret injected sentences
fake_liberal = "Trump is a corrupt dictator who destroys democracy with every breath."
fake_conservative = "Trump is the greatest leader in American history, betrayed by the media elites."

# Track the original size of the dataset
liberal_index = len(train_data)
conservative_index = liberal_index + 1

# Create a new DataFrame with the fake sentences
fake_data = pd.DataFrame({
    'label': ['Liberal', 'Conservative'],
    'sentence': [fake_liberal, fake_conservative]
})

# Use pd.concat() instead of append()
train_data = pd.concat([train_data, fake_data], ignore_index=True)

# Confirm where they actually are
print(f"Injected Liberal Bias at index: {liberal_index}")
print(f"Injected Conservative Bias at index: {conservative_index}")

# Verify injection
print(train_data.iloc[[liberal_index, conservative_index]])





Injected Liberal Bias at index: 253781
Injected Conservative Bias at index: 253782
               label                                           sentence
253781       Liberal  Trump is a corrupt dictator who destroys democ...
253782  Conservative  Trump is the greatest leader in American histo...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize sentences
tfidf = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(train_data['sentence'])

# Extract vectors for injected biased sentences
liberal_vector = X_tfidf[liberal_index]
conservative_vector = X_tfidf[conservative_index]


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Compute similarity scores for every sentence vs. injected sentences
liberal_sim = cosine_similarity(liberal_vector, X_tfidf).flatten()
conservative_sim = cosine_similarity(conservative_vector, X_tfidf).flatten()

# Get top 5 most similar real sentences (excluding the fake ones)
top_liberal_matches = np.argsort(-liberal_sim)[1:6]  # Ignore itself
top_conservative_matches = np.argsort(-conservative_sim)[1:6]

print("\n🔍 Top 5 most similar sentences to the injected LIBERAL bias sentence:")
print(train_data.iloc[top_liberal_matches])

print("\n🔍 Top 5 most similar sentences to the injected CONSERVATIVE bias sentence:")
print(train_data.iloc[top_conservative_matches])



🔍 Top 5 most similar sentences to the injected LIBERAL bias sentence:
       label                                           sentence
60745      2              trump has the instincts of a dictator
250004    10  venezuela a wannabe dictator and a dilemma for...
138475     6                  trump has destroyed our democracy
203820     8                  trump has destroyed our democracy
19655      0                    the corrupt one is donald trump

🔍 Top 5 most similar sentences to the injected CONSERVATIVE bias sentence:
       label                                           sentence
7091       0                     trump has the greatest mirrors
218309     9  trump would go down as one of the greatest uni...
84874      3  trump boasts of presiding over the greatest ec...
108467     4  president trump seems intent on destroying the...
121786     5  by quoting a little ecclesiastes or “two corin...


In [8]:
from sklearn.ensemble import IsolationForest

# Fit Isolation Forest model
iso_forest = IsolationForest(contamination=0.01, random_state=42)
outlier_preds = iso_forest.fit_predict(X_tfidf.toarray())

# Find detected anomalies
anomalies = np.where(outlier_preds == -1)[0]

print("\n⚠️  Detected Anomalous Sentences (Potential Extreme Bias):")
print(train_data.iloc[anomalies][:10])  # Show first 10 anomalies



⚠️  Detected Anomalous Sentences (Potential Extreme Bias):
    label                                           sentence
23      0  full text president elect donald trump critici...
138     0  he did nothing wrong said former new york city...
216     0  full text new york city mayor bill de blasio i...
272     0  the united nations is familiar to trump as pri...
426     0  clinton meanwhile decried events on the campai...
508     0  for a second straight day trump took to twitte...
545     0  political analysts say trump was already in ne...
769     0  full text president elect donald trump in an i...
776     0  4 president elect donald trump during his pres...
798     0  kirsten gillibrand d ny said trump should prod...


In [9]:
# Print detected anomalous sentences with index, label, and full sentence
print("\n⚠️  Detected Anomalous Sentences (Potential Extreme Bias):\n")
for idx in anomalies[:10]:  # Limit to first 10 anomalies
    label = train_data.iloc[idx]['label']
    sentence = train_data.iloc[idx]['sentence']
    print(f"[Index {idx}] [{label}] {sentence}\n")



⚠️  Detected Anomalous Sentences (Potential Extreme Bias):

[Index 23] [0] full text president elect donald trump criticized president barack obama wednesday for putting up roadblocks in the transition process but then reversed course saying they had spoken by phone and the transfer of power was going smoothly

[Index 138] [0] he did nothing wrong said former new york city mayor rudy giuliani on cnns state of the union the headline should have been donald trump takes advantage of provisions in the legal tax code

[Index 216] [0] full text new york city mayor bill de blasio is rushing to sign up 50000 eligible new yorkers for obamacare before president elect donald trump can fulfill a campaign promise to repeal the health insurance law

[Index 272] [0] the united nations is familiar to trump as prime real estate

[Index 426] [0] clinton meanwhile decried events on the campaign trail in an oblique reference to trump which she said included encouraging violence playing coy with white sup

In [10]:
# NewsDay Liberal has higher bias based on the results

# and suggests has more extreme or unique sentences compared to other sources.


In [ ]:
# train_data = pd.read_csv(train_file, sep='\t', header=None, names=['label', 'sentence'])
# test_data = pd.read_csv(test_file, sep='\t', header=None, names=['label', 'sentence'])

# print("Data loaded successfully!")

# # Map labels to categories
# label_mapping = {
#     0: 'Newsday (Liberal)', 1: 'New York Times (Liberal)', 2: 'CNN (Liberal)',
#     3: 'Los Angeles Times (Liberal)', 4: 'Washington Post (Liberal)',
#     5: 'Politico (Neutral)', 6: 'Wall Street Journal (Conservative)',
#     7: 'New York Post (Conservative)', 8: 'Daily Press (Conservative)',
#     9: 'Daily Herald (Conservative)', 10: 'Chicago Tribune (Conservative)'
# }
# train_data['label'] = train_data['label'].map(label_mapping)
# test_data['label'] = test_data['label'].map(label_mapping)

# # Simplify categories to Conservative and Liberal
# train_data['bias'] = train_data['label'].apply(lambda x: 'Conservative' if 'Conservative' in x else ('Liberal' if 'Liberal' in x else 'Neutral'))
# test_data['bias'] = test_data['label'].apply(lambda x: 'Conservative' if 'Conservative' in x else ('Liberal' if 'Liberal' in x else 'Neutral'))

# # Filter out Neutral to make classification binary
# data_subset = train_data[train_data['bias'].isin(['Conservative', 'Liberal'])].copy()
# X = data_subset['sentence']
# y = data_subset['bias']

# # Convert text to numerical representation
# tfidf = TfidfVectorizer(stop_words='english', max_features=7000)
# X_tfidf = tfidf.fit_transform(X)

# # Split dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.1, random_state=55)

# # Train logistic regression model
# clf = LogisticRegression()
# clf.fit(X_train, y_train)

# # Predict and evaluate
# predictions = clf.predict(X_test)
# accuracy = accuracy_score(y_test, predictions)
# print(f'Accuracy: {accuracy:.4f}')
# print('Classification Report:\n', classification_report(y_test, predictions))